In [ ]:
def load_data(filepath):
    with open(filepath, 'r') as f:
        data = f.read()
    return data

# Usage
data = load_data('/content/user-wallet-transactions.json')


In [ ]:
import json
import argparse
import pandas as pd
import numpy as np
from datetime import datetime


def load_data(input_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    return pd.json_normalize(data)


def feature_engineering(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['actionData.amount'] = pd.to_numeric(df['actionData.amount'], errors='coerce')

    features = []
    for wallet, g in df.groupby('userWallet'):
        f = {'wallet': wallet}

        f['n_tx'] = len(g)
        f['n_deposit'] = sum(g['action'] == 'deposit')
        f['n_borrow'] = sum(g['action'] == 'borrow')
        f['n_repay'] = sum(g['action'] == 'repay')
        f['n_liquidation'] = sum(g['action'] == 'liquidationcall')
        f['unique_assets'] = g['actionData.assetSymbol'].nunique()

        amounts = g.groupby('action')['actionData.amount'].agg(['mean', 'max', 'std']).unstack().fillna(0)
        if isinstance(amounts, pd.Series):
            for col, value in amounts.items():
                f[f'{col}'] = value
        else:
            for col in amounts.index:
                for stat in amounts.columns:
                    f[f'{col}_{stat}'] = amounts.loc[col, stat]


        g_sorted = g.sort_values('timestamp')
        if len(g_sorted) > 1:
            diffs = g_sorted['timestamp'].diff().dt.total_seconds()
            f['avg_time_diff'] = diffs[1:].mean()
        else:
            f['avg_time_diff'] = np.nan

        f['time_since_last'] = (datetime.utcnow() - g_sorted['timestamp'].iloc[-1]).total_seconds()

        total_borrow = g.loc[g['action'] == 'borrow', 'actionData.amount'].sum()
        total_deposit = g.loc[g['action'] == 'deposit', 'actionData.amount'].sum()
        total_repay = g.loc[g['action'] == 'repay', 'actionData.amount'].sum()

        f['borrow_deposit_ratio'] = total_borrow / total_deposit if total_deposit > 0 else 0
        f['repay_borrow_ratio'] = total_repay / total_borrow if total_borrow > 0 else 0

        features.append(f)
    return pd.DataFrame(features)


def compute_score(features):
    features['score_raw'] = (
        (1 - features['n_liquidation'] / (features['n_tx']+1)) * 0.4 +
        (1 - abs(features['borrow_deposit_ratio'] - 0.5)) * 0.3 +
        (features['repay_borrow_ratio']) * 0.3
    )

    features['score_raw'] = features['score_raw'].fillna(0)

    min_score = features['score_raw'].min()
    max_score = features['score_raw'].max()
    features['credit_score'] = 1000 * (features['score_raw'] - min_score) / (max_score - min_score + 1e-6)
    features['credit_score'] = features['credit_score'].round(0).astype(int)
    return features[['wallet', 'credit_score']]


def main(input_file, output_file):
    df = load_data(input_file)
    feats = feature_engineering(df)
    scores = compute_score(feats)
    scores.to_csv(output_file, index=False)
    print(f"Saved scores to {output_file}")


if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--input', required=True, help='Path to input json file')
    # parser.add_argument('--output', required=True, help='Path to output csv file')
    # args = parser.parse_args()

    # main(args.input, args.output)
    main('/content/user-wallet-transactions.json', '/content/user-wallet-scores.csv')

Saved scores to /content/user-wallet-scores.csv


In [ ]:
import json
import pandas as pd

def load_data(input_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    return pd.json_normalize(data)

df = load_data('/content/user-wallet-transactions.json')
print(df.columns)
display(df.head())

Index(['userWallet', 'network', 'protocol', 'txHash', 'logId', 'timestamp',
       'blockNumber', 'action', '__v', '_id.$oid', 'actionData.type',
       'actionData.amount', 'actionData.assetSymbol',
       'actionData.assetPriceUSD', 'actionData.poolId', 'actionData.userId',
       'createdAt.$date', 'updatedAt.$date', 'actionData.toId',
       'actionData.borrowRateMode', 'actionData.borrowRate',
       'actionData.variableTokenDebt', 'actionData.stableTokenDebt',
       'actionData.callerId', 'actionData.useATokens', 'actionData.repayerId',
       'actionData.liquidatorId', 'actionData.collateralAmount',
       'actionData.collateralAssetPriceUSD', 'actionData.principalAmount',
       'actionData.borrowAssetPriceUSD', 'actionData.collateralReserveId',
       'actionData.collateralReserveSymbol', 'actionData.principalReserveId',
       'actionData.principalReserveSymbol'],
      dtype='object')


,userWallet,network,protocol,txHash,logId,timestamp,blockNumber,action,__v,_id.$oid,...,actionData.repayerId,actionData.liquidatorId,actionData.collateralAmount,actionData.collateralAssetPriceUSD,actionData.principalAmount,actionData.borrowAssetPriceUSD,actionData.collateralReserveId,actionData.collateralReserveSymbol,actionData.principalReserveId,actionData.principalReserveSymbol
0,0x00000000001accfa9cef68cf5371a23025b6d4b6,polygon,aave_v2,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,1629178166,1629178166,deposit,0,681d38fed63812d4655f571a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x000000000051d07a4fb3bd10121a343d85818da6,polygon,aave_v2,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,1621525013,1621525013,deposit,0,681aa70dd6df53021cc6f3c0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,1627118913,1627118913,deposit,0,681d04c2d63812d4654c733e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,1627773318,1627773318,deposit,0,681d133bd63812d46551b6ef,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x0000000000e189dd664b9ab08a33c4839953852c,polygon,aave_v2,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,1618845907,1618845907,redeemunderlying,0,681899e4ba49fc91cf2f4454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
